In [15]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys  import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [ ]:
def scroll_all(driver):
    global ready_count
    ready_count = 0
    body = driver.find_element(By.TAG_NAME, "body")
    
    for _ in range(100):  # 10번 페이지 다운
        body.send_keys(Keys.END)
        time.sleep(1)
    
    ready_count = 1


In [17]:
def main():
    global parse
    driver = webdriver.Chrome()

    # login(driver, user_id, passwd)
    scroll_all(driver)       
    # time.sleep(5) 
    # parse = get_page_source(driver)
    
    time.sleep(5)
    driver.close()
    return 

In [18]:
# driver = webdriver.Chrome()

# # login(driver, user_id, passwd)
# driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# # driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
# scroll_all(driver)

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# results = soup.find_all('div', {'class': 'JobCard_JobCard__Tb7pI'})
# # 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# # print(results)

# # return results
# print(results)

In [19]:
driver = webdriver.Chrome()

# login(driver, user_id, passwd)
driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
scroll_all(driver)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

results_a = soup.find_all('div',{'class':'JobCard_JobCard__Tb7pI'})
results0 = soup.find_all('span', {'class': 'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'})
results1 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz'})
results2 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz'})
# 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# print(results)
time.sleep(1)
driver.close()

# return results
print(results_a[0].a['href'])
print(results0[0])
print(results1[0])
print(results2[0])


/wd/267736
<span class="JobCard_JobCard__body__position__CyaGY wds-1mh3acf">소프트웨어 엔지니어</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz">폴라리스오피스</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz">서울 구로구 · 신입-경력 6년</span>


In [20]:
exportDF = pd.DataFrame(columns=['포지션','회사명','지역명','경력','링크'])

In [21]:
region = []
years = []

In [22]:
for i in range(len(results2)):
    region.append(results2[i].text.split('·')[0])
    years.append(results2[i].text.split('·')[1])

In [23]:
for i in range(len(results0)):
    results0[i] = results0[i].text
    results1[i] = results1[i].text
    results2[i] = results2[i].text
    results_a[i] = results_a[i].a['href']

In [24]:
exportDF['포지션']=results0
exportDF['회사명']=results1
exportDF['지역명']=region
exportDF['경력']=years
exportDF['링크']=results_a

In [52]:
exportDF.to_csv('source.csv', encoding='utf-8')

In [56]:
exportDF

,포지션,회사명,지역명,경력,링크
0,소프트웨어 엔지니어,폴라리스오피스,서울 구로구,신입-경력 6년,/wd/267736
1,시니어 개발자 (Rust),베링랩,서울 마포구,경력 5-10년,/wd/206209
2,Unity 개발자,인터엑스,서울 금천구,경력 5-10년,/wd/261266
3,Head of Architecture,인터엑스,서울 금천구,경력 10년 이상,/wd/256936
4,AI Model Optimization Engineer (CPU/GPU/NPU),인터엑스,울산 중구,경력 3-15년,/wd/252960
...,...,...,...,...,...
3212,DBMS 엔진 개발,큐브리드,서울 강남구,경력 1-15년,/wd/118825
3213,Windows Application 개발자,아진엑스텍,경기 군포시,경력 8-12년,/wd/113536
3214,친환경 올인원 에어케어시스템 PCB 및 펌웨어,엔트,경기 부천시,경력 5-10년,/wd/76159
3215,"PCB약품 개발, 제조업, 화학약품",엔알지엔씨,경기 수원시,신입,/wd/87790


In [ ]:
article_list = []
main_task = list()
required = list()
preferred = list()
benefit =list()

In [ ]:

for i in exportDF.index[:1]:
    print(i)
    driver = webdriver.Chrome()
    driver.get(f"{'https://www.wanted.co.kr' + exportDF.loc[i,'링크']}")
    try:
        # "상세정보 더 보기" 버튼 찾기 (버튼의 텍스트가 '상세정보 더 보기'일 경우)
        button = driver.find_element(By.XPATH, "//*[@id='__next']/main/div[1]/div/section/section/article[1]/div/button")
        
        # 버튼 클릭
        button.click()
        
        # 클릭 후 페이지가 갱신될 시간을 기다림
        time.sleep(1)
    except :
        pass
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    detail_div = soup.find('div',{'class':"JobDescription_JobDescription__paragraph__wrapper__G4CNd"})
    detail_div_list = detail_div.find_all('div',{'class':'JobDescription_JobDescription__paragraph__Lhegj'})
    main_task.append(detail_div_list[0])
    required.append(detail_div_list[1])
    preferred.append(detail_div_list[2])
    benefit.append(detail_div_list[3])
    
    time.sleep(2)
    driver.close()

In [50]:
len(article_list)

213

In [ ]:

# article_list.append(article.text)


[<div class="JobDescription_JobDescription__paragraph__Lhegj"><h3 class="wds-1y0suvb">주요업무</h3><span class="wds-wcfcu3"><span>• Docker 기반 오피스 솔루션 구축 및 유지보수<br/>• 기업 요청사항에 따른 솔루션 기능개발<br/><br/>[사용하는 기술 및 환경]<br/>• Docker, Spring Java, JPA, JavaScript, JSP, jQuery, Ajax, HTML5, CSS</span></span></div>,
 <div class="JobDescription_JobDescription__paragraph__Lhegj"><h3 class="wds-1y0suvb">자격요건</h3><span class="wds-wcfcu3"><span>[자격요건]<br/>• 신입 ~ 경력 6년 이하<br/>• Spring 기반 Java API 구현 및 내부 서비스 구현<br/>• HTML5, CSS3 웹 표준에 대한 이해<br/>• JS(ES6)/HTML/CSS를 이용해 웹 페이지 개발 가능하신 분<br/>• RDBMS(MariaDB) 사용 경험이 있으신 분<br/>• 고객사와 원활하게 소통이 가능하며 외근 및 출장 업무에 거부감이 없으신 분<br/><br/>[필요지식 및 기술]<br/>• Spring 기반 Java API 구현 및 내부 서비스 구현<br/>• HTML5, CSS3 웹 표준에 대한 이해<br/>• JavaScript에 대한 이해 및 활용<br/>• shell script에 대한 이해<br/><br/>[이런 분을 선호해요!]<br/>• 새로운 AI 기술과 트렌드에 대한 열린 자세로 끊임없이 학습·적용하며, 변화에 유연하게 대응하는 태도가 좋은 분<br/>• 협업 동료와의 원활한 커뮤니케이션을 바탕으로 협업을 이끌어내고, 요구사항을 정확히 파악하여 반영하는 태도가 좋은 분<br/>• 사용자 중심의 관점과 논리적 사고를 통해 문제를 파악하고 해결

In [ ]:
with open("test_txt.txt", "w", encoding="utf-8") as file:
    for i in article_list:
        file.write(i + "\n")